# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'03-03-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'03-03-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-03-04 05:24:24,33.93911,67.709953,55770,2446,49359,3965.0,Afghanistan,143.263233,4.385871
1,NaN,NaN,NaN,Albania,2021-03-04 05:24:24,41.15330,20.168300,109674,1856,72076,35742.0,Albania,3811.036208,1.692288
2,NaN,NaN,NaN,Algeria,2021-03-04 05:24:24,28.03390,1.659600,113593,2996,78524,32073.0,Algeria,259.042869,2.637486
3,NaN,NaN,NaN,Andorra,2021-03-04 05:24:24,42.50630,1.521800,10948,112,10560,276.0,Andorra,14169.416942,1.023018
4,NaN,NaN,NaN,Angola,2021-03-04 05:24:24,-11.20270,17.873900,20923,510,19509,904.0,Angola,63.661016,2.437509


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,55617,55646,55664,55680,55696,55707,55714,55733,55759,55770
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,101285,102306,103327,104313,105229,106215,107167,107931,108823,109674
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,112094,112279,112461,112622,112805,112960,113092,113255,113430,113593


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2433,2435,2436,2438,2442,2443,2443,2444,2446,2446
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1681,1696,1715,1736,1756,1775,1796,1816,1835,1856
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2964,2967,2970,2973,2977,2979,2983,2987,2991,2996


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,48895,48967,49086,49281,49285,49288,49333,49344,49347,49359
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,64318,65403,66309,67158,68007,68969,69773,70413,71173,72076
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,77225,77382,77537,77683,77842,77976,78098,78234,78377,78524


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6121,6143,6172,6203,6228,6248,6264,6270,6303,6313
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,19461,19554,19635,19670,19698,19714,19732,19758,19790,19856
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2074,2084,2095,2099,2106,2113,2115,2116,2124,2129


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,85,84,85,89,90,91,91,91,91,91
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,262,263,272,274,277,283,283,283,284,285
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,50,50,50,51,51,51,51,51,51,51


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-03-04 05:24:24,32.539527,-86.644082,6313,91,0,6222.0,"Autauga, Alabama, US",11299.647389,1.441470
689,1075.0,Lamar,Alabama,US,2021-03-04 05:24:24,33.779950,-88.096680,1337,33,0,1304.0,"Lamar, Alabama, US",9684.896777,2.468212
690,1077.0,Lauderdale,Alabama,US,2021-03-04 05:24:24,34.901719,-87.656247,8610,212,0,8398.0,"Lauderdale, Alabama, US",9285.121160,2.462253


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,28758753,518345,0
India,11139516,157346,10812044
Brazil,10718630,259271,9548315


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,28758753,518345,0,28240408,2021-03-04 05:24:24,37.936303,-91.379001
India,11139516,157346,10812044,170126,2021-03-04 05:24:24,23.088275,81.806127
Brazil,10718630,259271,9548315,911044,2021-03-04 05:24:24,-12.669522,-48.480493
Russia,4230707,85901,3810327,334479,2021-03-04 05:24:24,54.546312,62.120860
United Kingdom,4207120,124017,11741,4071362,2021-03-04 05:24:24,28.164647,-33.966498


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3581093,52975,0
Texas,2675712,44463,0
Florida,1924114,31267,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3581093,52975,0,3528118,2021-03-04 05:24:24,37.843962,-120.728594
Texas,2675712,44463,0,2631249,2021-03-04 05:24:24,31.660643,-98.653069
Florida,1924114,31267,0,1892847,2021-03-04 05:24:24,28.940755,-82.700744
New York,1663248,47935,0,1615313,2021-03-04 05:24:24,42.544151,-75.474183
Illinois,1191440,22853,0,1168587,2021-03-04 05:24:24,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1195913,21669,0
Arizona,Maricopa,512286,9172,0
Illinois,Cook,476094,9406,0
Florida,Miami-Dade,414776,5503,0
Texas,Harris,355065,5155,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1195913,21669,0,1174244,2021-03-04 05:24:24,34.308284,-118.228241,6037.0
Arizona,Maricopa,512286,9172,0,503114,2021-03-04 05:24:24,33.348359,-112.491815,4013.0
Illinois,Cook,476094,9406,0,466688,2021-03-04 05:24:24,41.841448,-87.816588,17031.0
Florida,Miami-Dade,414776,5503,0,409273,2021-03-04 05:24:24,25.611236,-80.551706,12086.0
Texas,Harris,355065,5155,0,349910,2021-03-04 05:24:24,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-01,55733,107931,113255,10889,20854,769,2112023,172216,28986,460849,...,4194289,58589,79926,1,139545,2448,185336,2310,79002,36115
2021-03-02,55759,108823,113430,10908,20882,769,2118676,172456,28996,462769,...,4200700,59171,79961,1,139934,2475,187309,2342,79557,36148
2021-03-03,55770,109674,113593,10948,20923,769,2126531,172816,29007,465322,...,4207120,60074,80035,1,140383,2482,189326,2363,80090,36179


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-01,2444,1816,2987,110,508,14,52077,3195,909,8574,...,123187,611,622,0,1348,35,2058,634,1098,1468
2021-03-02,2446,1835,2991,110,510,14,52192,3200,909,8605,...,123530,617,622,0,1353,35,2063,640,1104,1472
2021-03-03,2446,1856,2996,112,510,19,52453,3202,909,8625,...,124017,624,622,0,1358,35,2078,643,1108,1478


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-01,49344,70413,78234,10475,19400,307,1911338,163511,22914,432016,...,11646,50624,78496,1,131647,1876,167744,1435,75087,32905
2021-03-02,49347,71173,78377,10501,19410,307,1916936,163738,22917,433873,...,11662,51365,78513,1,132052,1898,168763,1435,75563,32989
2021-03-03,49359,72076,78524,10560,19509,309,1921589,163906,22920,435669,...,11741,51978,78587,1,132503,1898,169915,1435,76299,33392


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6121,6143,6172,6203,6228,6248,6264,6270,6303,6313
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1296,1306,1316,1317,1324,1325,1328,1328,1331,1337
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,8525,8541,8570,8581,8588,8594,8603,8607,8601,8610
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2798,2806,2812,2813,2822,2826,2829,2834,2836,2846
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,14809,14827,14883,14900,14926,14950,14961,14967,14980,15021


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-01,493769,58647,0,817821,322509,3573549,429839,282626,87080,49,...,775693,2660791,371492,15284,2646,577174,340708,132048,617700,54471
2021-03-02,494421,58802,0,818670,322949,3577966,430615,283128,87193,49,...,776337,2668091,371979,15372,2646,578559,341441,132184,618307,54527
2021-03-03,497154,58993,0,819954,323353,3581093,431670,283622,87425,49,...,777935,2675712,372708,15487,2695,580108,342236,132416,619087,54616


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-03-01        6270   19758    2116  2450   6102    1169   1968   13300   
2021-03-02        6303   19790    2124  2454   6106    1171   1975   13307   
2021-03-03        6313   19856    2129  2459   6229    1172   2011   13755   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-03-01         3413     1770  ...    2618    591     3022      676   
2021-03-02         3421     1771  ...    2620    590     3023      677   
2021-03-03         3423     1777  ...    2622    590     3027      678   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-01           3772  3357  2055          0      883    622  
2021-03-02           3774  3359  2062          0      885    623  
2021-03-03           3800  3364  2066          0      886    623  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-01,9931,300,0,15979,5250,52491,5952,7651,1422,0,...,11421,43909,1940,205,25,8783,4969,2300,7014,671
2021-03-02,9991,302,0,16060,5254,52798,5959,7658,1426,0,...,11436,44186,1949,206,25,8943,4988,2301,7050,682
2021-03-03,10029,303,0,16089,5261,52975,5970,7678,1440,0,...,11459,44463,1955,207,25,9326,5012,2309,7069,682


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-03-01          91     283      51   60    127      36     66     286   
2021-03-02          91     284      51   60    127      36     66     287   
2021-03-03          91     285      51   60    127      36     66     288   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-03-01          110       38  ...      26     11       27        7   
2021-03-02          111       39  ...      27     11       30        7   
2021-03-03          111       39  ...      27     11       30        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-01             34     9    12          0       26      5  
2021-03-02             35     9    12          0       26      5  
2021-03-03             36     9    12          0       26      5  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-01,0.000341,0.007129,0.001441,0.002117,0.002259,0.053425,0.002210,0.000918,0.000276,0.003067,...,0.001304,0.010260,0.000000,0.0,0.003084,0.000000,0.009389,0.010941,0.005959,0.000720
2021-03-02,0.000467,0.008265,0.001545,0.001745,0.001343,0.000000,0.003150,0.001394,0.000345,0.004166,...,0.001529,0.009934,0.000438,0.0,0.002788,0.011029,0.010646,0.013853,0.007025,0.000914
2021-03-03,0.000197,0.007820,0.001437,0.003667,0.001963,0.000000,0.003708,0.002087,0.000379,0.005517,...,0.001528,0.015261,0.000925,0.0,0.003209,0.002828,0.010768,0.008967,0.006700,0.000858


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-01,0.000409,0.011136,0.001341,0.000000,0.000000,0.000000,0.002155,0.000940,0.0,0.001519,...,0.000845,0.004934,0.0,NaN,0.002976,0.0,0.007835,0.000000,0.006416,0.003418
2021-03-02,0.000818,0.010463,0.001339,0.000000,0.003937,0.000000,0.002208,0.001565,0.0,0.003616,...,0.002784,0.009820,0.0,NaN,0.003709,0.0,0.002430,0.009464,0.005464,0.002725
2021-03-03,0.000000,0.011444,0.001672,0.018182,0.000000,0.357143,0.005001,0.000625,0.0,0.002324,...,0.003942,0.011345,0.0,NaN,0.003695,0.0,0.007271,0.004687,0.003623,0.004076


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-01,0.000223,0.009173,0.001741,0.002776,0.004037,0.019934,0.003316,0.000716,0.000218,0.003720,...,0.000258,0.011691,0.000000,0.0,0.006214,0.000000,0.004840,0.0,0.007906,0.007316
2021-03-02,0.000061,0.010793,0.001828,0.002482,0.000515,0.000000,0.002929,0.001388,0.000131,0.004298,...,0.001374,0.014637,0.000217,0.0,0.003076,0.011727,0.006075,0.0,0.006339,0.002553
2021-03-03,0.000243,0.012687,0.001876,0.005619,0.005100,0.006515,0.002427,0.001026,0.000131,0.004139,...,0.006774,0.011934,0.000943,0.0,0.003415,0.000000,0.006826,0.0,0.009740,0.012216


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-01,0.001048,0.006591,NaN,0.001272,0.000292,0.001860,0.003586,0.009573,0.003237,0.0,...,0.000889,0.002932,0.000692,0.005659,0.000000,0.001951,0.002752,0.001464,0.000539,0.001416
2021-03-02,0.001320,0.002643,NaN,0.001038,0.001364,0.001236,0.001805,0.001776,0.001298,0.0,...,0.000830,0.002744,0.001311,0.005758,0.000000,0.002400,0.002151,0.001030,0.000983,0.001028
2021-03-03,0.005528,0.003248,NaN,0.001568,0.001251,0.000874,0.002450,0.001745,0.002661,0.0,...,0.002058,0.002856,0.001960,0.007481,0.018519,0.002677,0.002328,0.001755,0.001262,0.001632


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-03-01      0.000958  0.001318  0.000473  0.000000  0.000820  0.000000   
2021-03-02      0.005263  0.001620  0.003781  0.001633  0.000656  0.001711   
2021-03-03      0.001587  0.003335  0.002354  0.002037  0.020144  0.000854   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-03-01      0.003570  0.001883  0.000880  0.000000  ...  0.001147   
2021-03-02      0.003557  0.000526  0.002344  0.000565  ...  0.000764   
2021-03-03      0.018228  0.033666  0.000585  0.003388  ...  0.000763   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-03-01      0.001695  0.000662  0.000000   0.003458  0.001791  0.000974   
2021-03-02     -0.001692  0.000331  0.001479   0.000530  0.000596  0.003406   
2021-03-03      0.000000  0.001323  0.001477   0.006889  0.001489  0.001940   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-03-01            NaN  0.002270  0.000000  
2021-03-02            NaN  0.002265  0.001608  
2021-03-03            NaN  0.001130  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-01,0.000201,0.034483,NaN,-0.000063,0.001335,0.006384,0.000168,0.003805,0.000000,NaN,...,0.000876,0.004852,0.002584,0.004902,0.0,0.027011,0.002623,0.000000,0.000000,0.000000
2021-03-02,0.006042,0.006667,NaN,0.005069,0.000762,0.005849,0.001176,0.000915,0.002813,NaN,...,0.001313,0.006309,0.004639,0.004878,0.0,0.018217,0.003824,0.000435,0.005133,0.016393
2021-03-03,0.003803,0.003311,NaN,0.001806,0.001332,0.003352,0.001846,0.002612,0.009818,NaN,...,0.002011,0.006269,0.003079,0.004854,0.0,0.042827,0.004812,0.003477,0.002695,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2021-03-01         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   
2021-03-02         0.0  0.003534     0.0  0.0    0.0     0.0    0.0  0.003497   
2021-03-03         0.0  0.003521     0.0  0.0    0.0     0.0    0.0  0.003484   

Province_State                      ...   Wyoming                            \
Admin2          Chambers  Cherokee  ...      Park Platte  Sheridan Sublette   
2021-03-01      0.000000  0.000000  ...  0.000000    0.0  0.000000      0.0   
2021-03-02      0.009091  0.026316  ...  0.038462    0.0  0.111111      0.0   
2021-03-03      0.000000  0.000000  ...  0.000000    0.0  0.000000      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-01       0.000000   0.0   0.0        NaN      0.0    0.0  
2021-03-02       0.029412   0.0   0.0        NaN      0.0    0.0  
2021-03-03       0.028571   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-01,0.000265,0.008144,0.001381,0.002052,0.001911,0.035963,0.002192,0.001245,0.000263,0.003951,...,0.001515,0.011560,0.000378,0.0,0.003016,0.002332,0.008817,0.008465,0.005915,0.000786
2021-03-02,0.000366,0.008204,0.001463,0.001898,0.001627,0.017981,0.002671,0.001319,0.000304,0.004059,...,0.001522,0.010747,0.000408,0.0,0.002902,0.006681,0.009731,0.011159,0.006470,0.000850
2021-03-03,0.000281,0.008012,0.001450,0.002783,0.001795,0.008991,0.003189,0.001703,0.000342,0.004788,...,0.001525,0.013004,0.000667,0.0,0.003055,0.004754,0.010250,0.010063,0.006585,0.000854


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-01,0.000400,0.011297,0.001238,0.000145,0.001848,0.000767,0.001596,0.001057,2.985138e-23,0.002134,...,0.001381,0.006043,3.071421e-09,NaN,0.002711,2.595132e-56,0.006974,0.002353,0.008238,0.002115
2021-03-02,0.000609,0.010880,0.001288,0.000072,0.002893,0.000384,0.001902,0.001311,1.492569e-23,0.002875,...,0.002083,0.007931,1.535711e-09,NaN,0.003210,1.297566e-56,0.004702,0.005908,0.006851,0.002420
2021-03-03,0.000305,0.011162,0.001480,0.009127,0.001446,0.178763,0.003451,0.000968,7.462845e-24,0.002599,...,0.003013,0.009638,7.678553e-10,NaN,0.003453,6.487830e-57,0.005986,0.005298,0.005237,0.003248


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-01,0.000533,0.010923,0.001726,0.002668,0.002470,0.033368,0.003209,0.000792,0.000196,0.003640,...,0.001874,0.011233,0.000523,0.0,0.003979,0.006615,0.004899,0.00012,0.010967,0.004743
2021-03-02,0.000297,0.010858,0.001777,0.002575,0.001493,0.016684,0.003069,0.001090,0.000164,0.003969,...,0.001624,0.012935,0.000370,0.0,0.003528,0.009171,0.005487,0.00006,0.008653,0.003648
2021-03-03,0.000270,0.011773,0.001826,0.004097,0.003297,0.011599,0.002748,0.001058,0.000147,0.004054,...,0.004199,0.012435,0.000656,0.0,0.003471,0.004585,0.006156,0.00003,0.009197,0.007932


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-01,0.001251,0.003596,NaN,0.001360,0.003127,0.001497,0.003041,0.005230,0.003458,1.484307e-106,...,0.001293,0.002535,0.001134,0.006474,0.000876,0.002490,0.002121,0.001950,0.000808,0.001238
2021-03-02,0.001286,0.003119,NaN,0.001199,0.002246,0.001366,0.002423,0.003503,0.002378,7.421533e-107,...,0.001062,0.002639,0.001222,0.006116,0.000438,0.002445,0.002136,0.001490,0.000895,0.001133
2021-03-03,0.003407,0.003184,NaN,0.001384,0.001748,0.001120,0.002437,0.002624,0.002519,3.710767e-107,...,0.001560,0.002748,0.001591,0.006798,0.009478,0.002561,0.002232,0.001623,0.001078,0.001383


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-03-01      0.002046  0.001249  0.001292  0.000534  0.000831  0.000933   
2021-03-02      0.003655  0.001434  0.002536  0.001084  0.000743  0.001322   
2021-03-03      0.002621  0.002385  0.002445  0.001561  0.010444  0.001088   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-03-01      0.002765  0.002463  0.001202  0.000657  ...  0.000836   
2021-03-02      0.003161  0.001495  0.001773  0.000611  ...  0.000800   
2021-03-03      0.010694  0.017581  0.001179  0.001999  ...  0.000782   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-03-01      0.001675  0.000725  0.000627   0.003123  0.001320  0.000759   
2021-03-02     -0.000009  0.000528  0.001053   0.001827  0.000958  0.002083   
2021-03-03     -0.000004  0.000926  0.001265   0.004358  0.001223  0.002011   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-03-01           -1.0  0.001225  0.000045  
2021-03-02           -1.0  0.001745  0.000826  
2021-03-03           -1.0  0.001438  0.000413  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-01,0.001573,0.017258,NaN,0.001424,-0.006896,0.005741,0.000693,0.002127,0.001656,NaN,...,0.001614,0.004650,0.004449,0.003314,7.947286e-08,0.024476,0.001852,0.000880,0.000684,0.000108
2021-03-02,0.003808,0.011962,NaN,0.003247,-0.003067,0.005795,0.000935,0.001521,0.002234,NaN,...,0.001464,0.005479,0.004544,0.004096,3.973643e-08,0.021347,0.002838,0.000657,0.002908,0.008251
2021-03-03,0.003805,0.007637,NaN,0.002526,-0.000867,0.004574,0.001390,0.002066,0.006026,NaN,...,0.001737,0.005874,0.003811,0.004475,1.986821e-08,0.032087,0.003825,0.002067,0.002802,0.004125


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-03-01      0.003662  0.004203  0.000648  0.000673  0.002016  0.002316   
2021-03-02      0.001831  0.003868  0.000324  0.000337  0.001008  0.001158   
2021-03-03      0.000915  0.003695  0.000162  0.000168  0.000504  0.000579   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-03-01      0.002045  0.000882  0.004232  0.000422  ...  0.000002   
2021-03-02      0.001023  0.002189  0.006662  0.013369  ...  0.019232   
2021-03-03      0.000511  0.002837  0.003331  0.006685  ...  0.009616   

Province_State                                                                \
Admin2                Platte  Sheridan      Sublette    Sweetwater     Teton   
2021-03-01      2.935649e-12  0.000011  2.399701e-18  1.456811e-08  0.000008   
2021-03-02      1.467824e-12  0.055561  1.199850e-18  1.470589e-02  0.000004   
2021-03-03      7.339121e-13  0.027780  5.999252e-19  2.163866e-02  0.000002   

Province_State                                                       
Admin2                 Uinta Unassigned      Washakie        Weston  
2021-03-01      3.892916e-13  -0.501961  1.490975e-10  3.166497e-09  
2021-03-02      1.946458e-13  -0.501961  7.454873e-11  1.583248e-09  
2021-03-03      9.732290e-14  -0.501961  3.727436e-11  7.916242e-10  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210303,AK,56605,NaN,NaN,NaN,totalTestsViral,1711018,26.0,1285.0,...,NaN,1,-1,06e88c66f6d974e25a99aebd3d4be26ac862a54b,0,0,0,0,0,NaN
1,20210303,AL,497154,107382.0,1915545.0,NaN,totalTestsPeopleViral,2305317,559.0,45723.0,...,NaN,38,24,8f55732c81965a1c1dc1b4ef9f6645acfcdba89d,0,0,0,0,0,NaN
2,20210303,AR,323353,68716.0,2455309.0,NaN,totalTestsViral,2709946,397.0,14826.0,...,NaN,7,31,e297cee71343869e59cab919378d3bc788b963a5,0,0,0,0,0,NaN
3,20210303,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,bd1d4cf933585280fbe35034f0acf22a22123a5a,0,0,0,0,0,NaN
4,20210303,AZ,819954,55423.0,3022710.0,NaN,totalTestsViral,7739361,1165.0,57697.0,...,NaN,29,106,d2fed645fc1dece5459540521db025c23c6fe682,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210303,AK,56605.0,NaN,NaN,NaN,totalTestsViral,1711018.0,26.0,1285.0,...,NaN,1,-1,06e88c66f6d974e25a99aebd3d4be26ac862a54b,0,0,0,0,0,NaN
1,20210303,AL,497154.0,107382.0,1915545.0,NaN,totalTestsPeopleViral,2305317.0,559.0,45723.0,...,NaN,38,24,8f55732c81965a1c1dc1b4ef9f6645acfcdba89d,0,0,0,0,0,NaN
2,20210303,AR,323353.0,68716.0,2455309.0,NaN,totalTestsViral,2709946.0,397.0,14826.0,...,NaN,7,31,e297cee71343869e59cab919378d3bc788b963a5,0,0,0,0,0,NaN
3,20210303,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,bd1d4cf933585280fbe35034f0acf22a22123a5a,0,0,0,0,0,NaN
4,20210303,AZ,819954.0,55423.0,3022710.0,NaN,totalTestsViral,7739361.0,1165.0,57697.0,...,NaN,29,106,d2fed645fc1dece5459540521db025c23c6fe682,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-03,AK,56605,NaN,NaN,NaN,totalTestsViral,1711018,26.0,1285.0,NaN,...,NaN,1,-1,06e88c66f6d974e25a99aebd3d4be26ac862a54b,0,0,0,0,0,NaN
2021-03-03,AL,497154,107382.0,1915545.0,NaN,totalTestsPeopleViral,2305317,559.0,45723.0,NaN,...,NaN,38,24,8f55732c81965a1c1dc1b4ef9f6645acfcdba89d,0,0,0,0,0,NaN
2021-03-03,AR,323353,68716.0,2455309.0,NaN,totalTestsViral,2709946,397.0,14826.0,165.0,...,NaN,7,31,e297cee71343869e59cab919378d3bc788b963a5,0,0,0,0,0,NaN
2021-03-03,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,bd1d4cf933585280fbe35034f0acf22a22123a5a,0,0,0,0,0,NaN
2021-03-03,AZ,819954,55423.0,3022710.0,NaN,totalTestsViral,7739361,1165.0,57697.0,381.0,...,NaN,29,106,d2fed645fc1dece5459540521db025c23c6fe682,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-03,AK,56605.0,NaN,NaN,NaN,totalTestsViral,1711018.0,26.0,1285.0,NaN,...,NaN,1,-1,06e88c66f6d974e25a99aebd3d4be26ac862a54b,0,0,0,0,0,NaN
2021-03-03,AL,497154.0,107382.0,1915545.0,NaN,totalTestsPeopleViral,2305317.0,559.0,45723.0,NaN,...,NaN,38,24,8f55732c81965a1c1dc1b4ef9f6645acfcdba89d,0,0,0,0,0,NaN
2021-03-03,AR,323353.0,68716.0,2455309.0,NaN,totalTestsViral,2709946.0,397.0,14826.0,165.0,...,NaN,7,31,e297cee71343869e59cab919378d3bc788b963a5,0,0,0,0,0,NaN
2021-03-03,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,bd1d4cf933585280fbe35034f0acf22a22123a5a,0,0,0,0,0,NaN
2021-03-03,AZ,819954.0,55423.0,3022710.0,NaN,totalTestsViral,7739361.0,1165.0,57697.0,381.0,...,NaN,29,106,d2fed645fc1dece5459540521db025c23c6fe682,0,0,0,0,0,NaN
